In [402]:
import sqlite3 as sqlt
import pandas as pd
from datetime import datetime
import dateutil.relativedelta as oper_date
import io
import requests as req
import numpy as np

In [18]:
querie = """select * from exports where "Cod. Mun" in {} and 
"NCM" like "6402%";"""
grendene_cidades = "(4307906, 4304804,  2312908, 2931350, 2304202, 2304400)"
alpargatas_cidades = "(2604007, 2504009, 3143302, 3530607)"


In [19]:
con = sqlt.connect("big_data2.db")
cursor = con.cursor()

In [601]:
def get_trimestre(string):
    #print(string)
    mes, ano = string.split("T")
    ano = "20" + ano
    dict_mes = {"1":3, "2":6, "3":9, "4":12}
    return datetime(int(ano), dict_mes[mes], 1)

def data_clean_db(dataframe):
    dict_nomes_colunas = {"month":"data","US$ de P1":"financeiro","Desc. Mun":"municipio",
                          "Kg Liq de P1":"peso","Qtd de P1":"quantidade"}
    dataframe.rename(columns=dict_nomes_colunas,inplace=True)
    dataframe.drop(axis=1,columns=['Cod. Mun', 'municipio', 'NCM', "Desc. NCM"],inplace=True)
    dataframe.data = dataframe.data.apply(
        lambda data: datetime.strptime(data.split()[0], "%Y-%m-%d"))
    dataframe.set_index("index", inplace=True)
    dataframe.financeiro = dataframe.financeiro.astype("int")
    dataframe.peso = dataframe.peso.astype("int") 
    
    return dataframe

def data_clean_xls(dataframe):
    dataframe.reset_index(inplace=True)
    grendene_exports_of = dataframe[["index","Volume exportado Grendene"]]
    grendene_exports_of.rename(columns={"index":"data","Volume exportado Grendene":"volume"},inplace=True)

    alpargatas_exports_of = dataframe[["Unnamed: 2","Volume exportado Alpargatas"]]
    alpargatas_exports_of.rename(columns={"Unnamed: 2":"data","Volume exportado Alpargatas":"volume"},inplace=True)
    grendene_exports_of.data = grendene_exports_of.data.apply(lambda data:data.date())
    alpargatas_exports_of.data = alpargatas_exports_of.data.apply(lambda data:data.date())
    
    alpargatas_exports_of.dropna(inplace=True)
    grendene_exports_of.volume = grendene_exports_of.volume.astype("int")
    alpargatas_exports_of.volume = alpargatas_exports_of.volume.astype("int")

    return (grendene_exports_of, alpargatas_exports_of)

In [243]:
grendene_db = pd.read_sql_query(querie.format(grendene_cidades), con)
alpargatas_db = pd.read_sql_query(querie.format(alpargatas_cidades), con)

grendene_db = data_clean_db(grendene_db)
alpargatas_db = data_clean_db(alpargatas_db)

In [247]:
grendene_db

,financeiro,peso,quantidade,data
index,,,,
571,247150,44329,66090,2003-01-01
634,4917,894,2784,2003-01-01
764,6328919,1126374,2831893,2003-01-01
19625,20160,4922,10080,2003-01-01
20383,3308,372,720,2003-01-01
20384,63372,2829,6120,2003-01-01
683,143018,22594,36348,2004-01-01
745,6364,540,1104,2004-01-01
925,7611,575,864,2004-01-01


In [24]:
exportacoes_oficiais = pd.read_excel("Exportação calçados.xlsx")
grendene_exports_of, alpargatas_exports_of = data_clean_xsls(exportacoes_oficiais)

/home/rafa/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/home/rafa/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/home/rafa/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [25]:
for index, linha in grendene_exports_of.iterrows():
    soma = grendene_db[grendene_db.data==linha["data"]].financeiro.sum()
    string = "data: {}\nvalor movimentado oficial: {}\nvalor movimentado no db:   {}\n\n"
    print(string.format(linha["data"], linha["volume"],soma))

data: 2009-03-01
valor movimentado oficial: 15722000
valor movimentado no db:   11944829


data: 2009-06-01
valor movimentado oficial: 8141000
valor movimentado no db:   7911437


data: 2009-09-01
valor movimentado oficial: 7724000
valor movimentado no db:   7493393


data: 2009-12-01
valor movimentado oficial: 16698000
valor movimentado no db:   13119399


data: 2010-03-01
valor movimentado oficial: 21201000
valor movimentado no db:   19620984


data: 2010-06-01
valor movimentado oficial: 10173000
valor movimentado no db:   10515877


data: 2010-09-01
valor movimentado oficial: 9213000
valor movimentado no db:   17392900


data: 2010-12-01
valor movimentado oficial: 14046000
valor movimentado no db:   23130423


data: 2011-03-01
valor movimentado oficial: 12832000
valor movimentado no db:   11603149


data: 2011-06-01
valor movimentado oficial: 7481000
valor movimentado no db:   6839422


data: 2011-09-01
valor movimentado oficial: 9313000
valor movimentado no db:   8500638


data: 20

In [17]:
for index, linha in alpargatas_exports_of.iterrows():
    soma = alpargatas_db[alpargatas_db.data==linha["data"]].financeiro.sum()
    string = "data: {}\nvalor movimentado oficial: {}\nvalor movimentado no db:   {}\n\n"
    print(string.format(linha["data"], linha["volume"],soma))

data: 2010-03-01
valor movimentado oficial: 6500000
valor movimentado no db:   6198785


data: 2010-06-01
valor movimentado oficial: 7500000
valor movimentado no db:   4336305


data: 2010-09-01
valor movimentado oficial: 6100000
valor movimentado no db:   5720937


data: 2010-12-01
valor movimentado oficial: 6100000
valor movimentado no db:   7886964


data: 2011-03-01
valor movimentado oficial: 7600000
valor movimentado no db:   5540689


data: 2011-06-01
valor movimentado oficial: 8300000
valor movimentado no db:   7246531


data: 2011-09-01
valor movimentado oficial: 6100000
valor movimentado no db:   6769557


data: 2011-12-01
valor movimentado oficial: 6100000
valor movimentado no db:   7740834


data: 2012-03-01
valor movimentado oficial: 7692000
valor movimentado no db:   9975289


data: 2012-06-01
valor movimentado oficial: 10484000
valor movimentado no db:   9916039


data: 2012-09-01
valor movimentado oficial: 5797000
valor movimentado no db:   12162386


data: 2012-12-01
va

In [235]:
grendene_db

,Cod. Mun,municipio,NCM,financeiro,peso,quantidade,data
index,,,,,,,
571,2304202,CRATO-CE,64022000,247150,44329,66090,2003-01-01
634,2304400,FORTALEZA-CE,64022000,4917,894,2784,2003-01-01
764,2312908,SOBRAL-CE,64022000,6328919,1126374,2831893,2003-01-01
19625,4304804,CARLOS BARBOSA-RS,64029900,20160,4922,10080,2003-01-01
20383,4307906,FARROUPILHA-RS,64022000,3308,372,720,2003-01-01
20384,4307906,FARROUPILHA-RS,64029900,63372,2829,6120,2003-01-01
683,2304202,CRATO-CE,64022000,143018,22594,36348,2004-01-01
745,2304400,FORTALEZA-CE,64021900,6364,540,1104,2004-01-01
925,2312908,SOBRAL-CE,64021900,7611,575,864,2004-01-01


In [252]:
g = grendene_db
anos = list(range(2003,2018))
trimestres = [3, 6, 9, 12]
balanco_mensal = pd.DataFrame(columns=["1M","2M","3M" ,"trimestre"])
mes3 = (lambda exportacao: exportacao[2] if len(exportacao) == 3 else 0)
for ano in anos:
    for trimestre in trimestres:
        trimestre_ = datetime(ano, trimestre, 1)
        tabela_tr = g[(g.data <= trimestre_) & 
                      (g.data > trimestre_ - oper_date.relativedelta(months=3))]
        #g -= tabela_tr
        exportacao = tabela_tr.groupby("data").quantidade.sum()
        balanco_mensal = balanco_mensal.append(
                    {"1M": exportacao[0],
                    "2M": exportacao[1],
                     "3M": mes3(exportacao),
                     "trimestre": trimestre_ }, ignore_index=True)        

In [253]:
balanco_mensal.set_index("trimestre",inplace=True)
balanco_mensal

,1M,2M,3M
trimestre,,,
2003-03-01,2917687,2990283,2452614
2003-06-01,2197177,1767084,661129
2003-09-01,1046892,1678653,0
2003-12-01,1970830,1059055,2594101
2004-03-01,3633704,3999037,3650149
2004-06-01,2617339,1367714,956113
2004-09-01,1145208,1710888,0
2004-12-01,2219661,1825165,2341841
2005-03-01,3243056,3573561,3017752


In [618]:
url = "http://ri.grendene.com.br/Arquivos/Grendene_Informacoes_Financeiras_Download.xls"
balanco_trimestral=pd.read_excel(url, encoding="latin-1",sheet_name="Volumes")

In [619]:
balanco_trimestral.drop(columns="Grendene S.A.",inplace=True)

In [620]:
#balanco_trimestral.iloc[[2,4],[1,2,4,6]]

In [621]:
dict_ = balanco_trimestral.iloc[[2,4],].iloc[0]

In [622]:
#balanco_trimestral = balanco_trimestral.iloc[[2,4],].rename(columns=dict_)

In [623]:
balanco_trimestral = balanco_trimestral.iloc[[2,4],]
balanco_trimestral.reset_index(inplace=True)
balanco_trimestral

,index,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90
0,NaN,2006.0,1T07,2T07,1S07,3T07,9M07,4T07,2S07,2007.0,...,2016.0,1T17,2T17,1S17,3T17,9M17,4T17,2S17,2017.0,1T18
1,NaN,31503.0,13152,6324,19476,7609,27085,12975,20584,40060.0,...,39962.0,11016,9228,20244,8733,28977,15994,24727,44971.0,10991


In [624]:
balanco_trimestral = balanco_trimestral.iloc[[0,1]].reset_index().iloc[:,2:]
balanco_trimestral

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90
0,2006.0,1T07,2T07,1S07,3T07,9M07,4T07,2S07,2007.0,1T08,...,2016.0,1T17,2T17,1S17,3T17,9M17,4T17,2S17,2017.0,1T18
1,31503.0,13152,6324,19476,7609,27085,12975,20584,40060.0,18631,...,39962.0,11016,9228,20244,8733,28977,15994,24727,44971.0,10991


In [625]:
balanco_trimestral = balanco_trimestral.transpose().rename(columns={0:"trimestre",1:"exportacao"})

In [626]:
balanco_trimestral.info()

<class 'pandas.core.frame.DataFrame'>
Index: 90 entries, Unnamed: 1 to Unnamed: 90
Data columns (total 2 columns):
trimestre     90 non-null object
exportacao    90 non-null object
dtypes: object(2)
memory usage: 2.1+ KB


In [627]:
balanco_trimestral = balanco_trimestral.reset_index().drop(axis=1, columns="index")

In [628]:
balanco_trimestral.trimestre = balanco_trimestral.trimestre.astype("str")

In [629]:
balanco_trimestral

,trimestre,exportacao
0,2006.0,31503
1,1T07,13152
2,2T07,6324
3,1S07,19476
4,3T07,7609
5,9M07,27085
6,4T07,12975
7,2S07,20584
8,2007.0,40060
9,1T08,18631


In [630]:
balanco_trimestral = balanco_trimestral[balanco_trimestral.trimestre.str[1] == "T"]
balanco_trimestral.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45 entries, 1 to 89
Data columns (total 2 columns):
trimestre     45 non-null object
exportacao    45 non-null object
dtypes: object(2)
memory usage: 1.1+ KB


In [631]:
balanco_trimestral.trimestre = balanco_trimestral.trimestre.apply(
    lambda string:get_trimestre(string))

/home/rafa/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [632]:
balanco_trimestral.exportacao = balanco_trimestral.exportacao.apply(
    lambda exportacao:exportacao*1000)

/home/rafa/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [616]:
balanco_mensal.iloc[23]

1M    2770032
2M    2386557
3M    5822909
Name: 2008-12-01 00:00:00, dtype: object

In [ ]:
6786
6744934

In [617]:
balanco_trimestral

1     13152000
2      6324000
4      7609000
6     12975000
9     18631000
10     6786000
12     9142000
14    13285000
17    15722000
18     8141000
20     7724000
22    16698000
25    21201000
26    10173000
28     9213000
30    14046000
33    12832000
34     7481000
36     9313000
38    12903000
41    14244000
42     7796000
44     8796000
46    14510000
49    15063000
50     9662000
52    10930000
54    14873000
57    15780000
58     9144000
60    11395000
62    15874000
65    12419000
66     8645000
68    10107000
70    14755000
73    11159000
74     7374000
76    10220000
78    11209000
81    11016000
82     9228000
84     8733000
86    15994000
89    10991000
Name: exportacao, dtype: int64